In [1]:
# tylko nowe cechy
# liczba nowych cech to połowa oryginalnych cech
# sqrt(n) zaokrąglony w góre
# klasyfikatory - zafiksowac sensowne ustawienia
# wagi sumowane do 1
# lecim po wszystkich zbiorach czemu nie
# k-fold cross validation
from multi_imbalance.datasets import load_datasets
import experiment as ex
from sklearn.model_selection import StratifiedKFold
from collections import Counter, defaultdict
from multi_imbalance.utils.metrics import gmean_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

/home/dhorna/anaconda3/envs/mgr/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
datasets = load_datasets()

In [3]:
config = ex.config_calculation_strategy1(datasets)

In [4]:
config

{'1czysty-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 4,
   'units_3rd_layer': 2,
   'units_latent_layer': 1},
  'weighted_triplet_loss': True},
 '2delikatne-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 4,
   'units_3rd_layer': 2,
   'units_latent_layer': 1},
  'weighted_triplet_loss': True},
 '3mocniej-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 4,
   'units_3rd_layer': 2,
   'units_latent_layer': 1},
  'weighted_triplet_loss': True},
 '4delikatne-bezover-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 4,
   'units_3rd_layer': 2,
   'units_latent_layer': 1},
  'weighted_triplet_loss': True},
 'balance-scale': {'nn_config': {'units_1st_layer': 4,
   'units_2nd_layer': 8,
   'units_3rd_layer': 4,
   'units_latent_layer': 2},
  'weighted_triplet_loss': True},
 'cleveland': {'nn_config': {'units_1st_layer': 13,
   'units_2nd_layer': 26,
   'units_3rd_layer': 13,
   'units_latent_layer': 6},
  'weighted_triplet_

In [5]:
def instantiate_classifiers():
    classifiers = {
            "RF": RandomForestClassifier(max_depth=6, random_state=0),
            "KNN3": KNeighborsClassifier(n_neighbors=3),
            "KNN1": KNeighborsClassifier(n_neighbors=3) # FIXME
        } 
    return classifiers

In [6]:
result_dict = defaultdict(list)

# for dataset_name in list(datasets.keys())[5:6]:
for dataset_name in list(datasets.keys()):
    print(dataset_name)
    result_dict["dataset"].append(dataset_name)
    X, y = datasets[dataset_name]['data'], datasets[dataset_name]['target']
    skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    n_splits = skf.get_n_splits(X, y)
    fold_result_dict = defaultdict(list)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        weights = ex.weights_calculation_strategy1(X_train, y_train)
        nn_config =  config[dataset_name]['nn_config']
        # pretrain ANN triplets
        embeddings_train, embeddings_test = ex.train_triplets(X_train, y_train, X_test, y_test, weights,nn_config)
        X_train_new = np.hstack([X_train, embeddings_train])
        X_test_new = np.hstack([X_test, embeddings_test])
        
        classifiers = instantiate_classifiers()
        for clf_name, clf in classifiers.items():
            clf.fit(X_train, y_train)
            preds = clf.predict(X_test)

            gmean = gmean_score(y_test, preds)
            f1 = f1_score(y_test, preds, average='macro')
            fold_result_dict[f"{clf_name}-f1-old-features"].append(f1)
            fold_result_dict[f"{clf_name}-gmean-old-features"].append(gmean)
            
        classifiers = instantiate_classifiers()
        for clf_name, clf in classifiers.items():
            clf.fit(X_train_new, y_train)
            preds = clf.predict(X_test_new)

            gmean = gmean_score(y_test, preds)
            f1 = f1_score(y_test, preds, average='macro')
            fold_result_dict[f"{clf_name}-f1-old+new-features"].append(f1)
            fold_result_dict[f"{clf_name}-gmean-old+new-features"].append(gmean)
            
        classifiers = instantiate_classifiers()
        for clf_name, clf in classifiers.items():
            clf.fit(embeddings_train, y_train)
            preds = clf.predict(embeddings_test)

            gmean = gmean_score(y_test, preds)
            f1 = f1_score(y_test, preds, average='macro')
            fold_result_dict[f"{clf_name}-f1-new-features"].append(f1)
            fold_result_dict[f"{clf_name}-gmean-new-features"].append(gmean)
    for k in fold_result_dict:
        result_dict[k].append(np.mean(fold_result_dict[k]))
        
            
            #f1, gmean
    # mean over folds
    # wiersze: datasety, 
    # kolumny -> klasyfikatory (przed dodaniem cech, po dodaniu, na samych nowych cechach)

1czysty-cut
Train Epoch: 1 [0/960 (0%)]	Loss: 12.860626
Train Epoch: 1 [60/960 (67%)]	Loss: 5.549721

Test set: Average loss: 31.2417

Train Epoch: 2 [0/960 (0%)]	Loss: 4.965275
Train Epoch: 2 [60/960 (67%)]	Loss: 2.769786

Test set: Average loss: 16.1506

Train Epoch: 3 [0/960 (0%)]	Loss: 2.738162
Train Epoch: 3 [60/960 (67%)]	Loss: 0.979109

Test set: Average loss: 9.6313

Train Epoch: 4 [0/960 (0%)]	Loss: 2.267468
Train Epoch: 4 [60/960 (67%)]	Loss: 0.573120

Test set: Average loss: 6.7979

Train Epoch: 5 [0/960 (0%)]	Loss: 0.932415
Train Epoch: 5 [60/960 (67%)]	Loss: 0.433422

Test set: Average loss: 5.2656

Train Epoch: 6 [0/960 (0%)]	Loss: 0.688416
Train Epoch: 6 [60/960 (67%)]	Loss: 0.541368

Test set: Average loss: 4.3802

Train Epoch: 7 [0/960 (0%)]	Loss: 0.810674
Train Epoch: 7 [60/960 (67%)]	Loss: 0.443741

Test set: Average loss: 3.8131

Train Epoch: 8 [0/960 (0%)]	Loss: 0.434141
Train Epoch: 8 [60/960 (67%)]	Loss: 0.358731

Test set: Average loss: 3.2504

Train Epoch: 9 [0

100%|██████████| 240/240 [00:00<00:00, 2494.72it/s]


Train Epoch: 1 [0/960 (0%)]	Loss: 6.405216
Train Epoch: 1 [60/960 (67%)]	Loss: 2.639739

Test set: Average loss: 17.5108

Train Epoch: 2 [0/960 (0%)]	Loss: 1.422950
Train Epoch: 2 [60/960 (67%)]	Loss: 1.057978

Test set: Average loss: 10.1181

Train Epoch: 3 [0/960 (0%)]	Loss: 1.023652
Train Epoch: 3 [60/960 (67%)]	Loss: 0.237370

Test set: Average loss: 6.3063

Train Epoch: 4 [0/960 (0%)]	Loss: 0.644238
Train Epoch: 4 [60/960 (67%)]	Loss: 0.670643

Test set: Average loss: 4.6669

Train Epoch: 5 [0/960 (0%)]	Loss: 0.716177
Train Epoch: 5 [60/960 (67%)]	Loss: 0.560032

Test set: Average loss: 3.5278

Train Epoch: 6 [0/960 (0%)]	Loss: 0.299213
Train Epoch: 6 [60/960 (67%)]	Loss: 0.369152

Test set: Average loss: 3.0116

Train Epoch: 7 [0/960 (0%)]	Loss: 0.487881
Train Epoch: 7 [60/960 (67%)]	Loss: 0.323842

Test set: Average loss: 2.4733

Train Epoch: 8 [0/960 (0%)]	Loss: 0.261736
Train Epoch: 8 [60/960 (67%)]	Loss: 0.233508

Test set: Average loss: 2.2114

Train Epoch: 9 [0/960 (0%)]	Lo

 34%|███▍      | 331/960 [00:00<00:00, 3308.29it/s]


Test set: Average loss: 1.4018



...

100%|██████████| 43/43 [00:00<00:00, 2924.09it/s]


Train Epoch: 1 [0/172 (0%)]	Loss: 0.124736

Test set: Average loss: 0.4398

Train Epoch: 2 [0/172 (0%)]	Loss: 0.259332

Test set: Average loss: 0.4566

Train Epoch: 3 [0/172 (0%)]	Loss: 0.178133

Test set: Average loss: 0.5081

Train Epoch: 4 [0/172 (0%)]	Loss: 0.136608

Test set: Average loss: 0.3830

Train Epoch: 5 [0/172 (0%)]	Loss: 0.139268

Test set: Average loss: 0.3978

Train Epoch: 6 [0/172 (0%)]	Loss: 0.139688

Test set: Average loss: 0.3926

Train Epoch: 7 [0/172 (0%)]	Loss: 0.143569

Test set: Average loss: 0.3857

Train Epoch: 8 [0/172 (0%)]	Loss: 0.143784

Test set: Average loss: 0.3354

Train Epoch: 9 [0/172 (0%)]	Loss: 0.095764

Test set: Average loss: 0.4215

Train Epoch: 10 [0/172 (0%)]	Loss: 0.084229

Test set: Average loss: 0.3096

Train Epoch: 11 [0/172 (0%)]	Loss: 0.059169

Test set: Average loss: 0.3828

Train Epoch: 12 [0/172 (0%)]	Loss: 0.086921

Test set: Average loss: 0.4198

Train Epoch: 13 [0/172 (0%)]	Loss: 0.088972

Test set: Average loss: 0.2835

Train Ep

  0%|          | 0/43 [00:00<?, ?it/s]


Test set: Average loss: 0.2991



100%|██████████| 43/43 [00:00<00:00, 2486.80it/s]


In [7]:
results_df = pd.DataFrame.from_dict(result_dict)

In [8]:
results_df

,dataset,RF-f1-old-features,RF-gmean-old-features,KNN3-f1-old-features,KNN3-gmean-old-features,KNN1-f1-old-features,KNN1-gmean-old-features,RF-f1-old+new-features,RF-gmean-old+new-features,KNN3-f1-old+new-features,KNN3-gmean-old+new-features,KNN1-f1-old+new-features,KNN1-gmean-old+new-features,RF-f1-new-features,RF-gmean-new-features,KNN3-f1-new-features,KNN3-gmean-new-features,KNN1-f1-new-features,KNN1-gmean-new-features
0,1czysty-cut,0.919858,0.917711,0.950533,0.954519,0.950533,0.954519,0.926485,0.922971,0.950533,0.954519,0.950533,0.954519,0.332209,0.045521,0.397392,0.219791,0.397392,0.219791
1,2delikatne-cut,0.784165,0.741927,0.728790,0.688025,0.728790,0.688025,0.773889,0.717699,0.728790,0.688025,0.728790,0.688025,0.474822,0.082323,0.442137,0.228260,0.442137,0.228260
2,3mocniej-cut,0.598026,0.466120,0.496819,0.382030,0.496819,0.382030,0.573306,0.407404,0.496819,0.382030,0.496819,0.382030,0.364703,0.058684,0.384577,0.092762,0.384577,0.092762
3,4delikatne-bezover-cut,0.834079,0.821607,0.819202,0.798464,0.819202,0.798464,0.832779,0.819098,0.819202,0.798464,0.819202,0.798464,0.376862,0.060931,0.427468,0.306155,0.427468,0.306155
4,balance-scale,0.607284,0.096011,0.575163,0.089997,0.575163,0.089997,0.781383,0.686691,0.582156,0.091201,0.582156,0.091201,0.640312,0.428371,0.699749,0.647014,0.699749,0.647014
5,cleveland,0.293520,0.074883,0.194154,0.017399,0.194154,0.017399,0.314496,0.085297,0.194264,0.017399,0.194264,0.017399,0.264488,0.058665,0.261374,0.060507,0.261374,0.060507
6,cleveland_v2,0.298832,0.049221,0.228293,0.010998,0.228293,0.010998,0.321649,0.056971,0.228293,0.010998,0.228293,0.010998,0.305080,0.037275,0.283716,0.042900,0.283716,0.042900
7,cmc,0.524315,0.503251,0.481749,0.459991,0.481749,0.459991,0.523067,0.500289,0.460201,0.438607,0.460201,0.438607,0.491538,0.466748,0.450629,0.422081,0.450629,0.422081
8,dermatology,0.966571,0.957366,0.866114,0.860522,0.866114,0.860522,0.956508,0.949744,0.878055,0.871628,0.878055,0.871628,0.928054,0.920477,0.938882,0.932382,0.938882,0.932382
9,glass,0.674466,0.401595,0.613173,0.434651,0.613173,0.434651,0.654870,0.404507,0.615538,0.436438,0.615538,0.436438,0.373892,0.093753,0.425459,0.124890,0.425459,0.124890


In [13]:
results_df.filter(regex=("dataset|RF-f1*"))

,dataset,RF-f1-old-features,RF-f1-old+new-features,RF-f1-new-features
0,1czysty-cut,0.919858,0.926485,0.332209
1,2delikatne-cut,0.784165,0.773889,0.474822
2,3mocniej-cut,0.598026,0.573306,0.364703
3,4delikatne-bezover-cut,0.834079,0.832779,0.376862
4,balance-scale,0.607284,0.781383,0.640312
5,cleveland,0.293520,0.314496,0.264488
6,cleveland_v2,0.298832,0.321649,0.305080
7,cmc,0.524315,0.523067,0.491538
8,dermatology,0.966571,0.956508,0.928054
9,glass,0.674466,0.654870,0.373892


In [14]:
results_df.filter(regex=("dataset|RF-gmean*"))

,dataset,RF-gmean-old-features,RF-gmean-old+new-features,RF-gmean-new-features
0,1czysty-cut,0.917711,0.922971,0.045521
1,2delikatne-cut,0.741927,0.717699,0.082323
2,3mocniej-cut,0.466120,0.407404,0.058684
3,4delikatne-bezover-cut,0.821607,0.819098,0.060931
4,balance-scale,0.096011,0.686691,0.428371
5,cleveland,0.074883,0.085297,0.058665
6,cleveland_v2,0.049221,0.056971,0.037275
7,cmc,0.503251,0.500289,0.466748
8,dermatology,0.957366,0.949744,0.920477
9,glass,0.401595,0.404507,0.093753


In [15]:
results_df.filter(regex=("dataset|KNN3-f1*"))

,dataset,KNN3-f1-old-features,KNN3-f1-old+new-features,KNN3-f1-new-features
0,1czysty-cut,0.950533,0.950533,0.397392
1,2delikatne-cut,0.728790,0.728790,0.442137
2,3mocniej-cut,0.496819,0.496819,0.384577
3,4delikatne-bezover-cut,0.819202,0.819202,0.427468
4,balance-scale,0.575163,0.582156,0.699749
5,cleveland,0.194154,0.194264,0.261374
6,cleveland_v2,0.228293,0.228293,0.283716
7,cmc,0.481749,0.460201,0.450629
8,dermatology,0.866114,0.878055,0.938882
9,glass,0.613173,0.615538,0.425459


In [16]:
results_df.filter(regex=("dataset|KNN3-gmean*"))

,dataset,KNN3-gmean-old-features,KNN3-gmean-old+new-features,KNN3-gmean-new-features
0,1czysty-cut,0.954519,0.954519,0.219791
1,2delikatne-cut,0.688025,0.688025,0.228260
2,3mocniej-cut,0.382030,0.382030,0.092762
3,4delikatne-bezover-cut,0.798464,0.798464,0.306155
4,balance-scale,0.089997,0.091201,0.647014
5,cleveland,0.017399,0.017399,0.060507
6,cleveland_v2,0.010998,0.010998,0.042900
7,cmc,0.459991,0.438607,0.422081
8,dermatology,0.860522,0.871628,0.932382
9,glass,0.434651,0.436438,0.124890
